In [66]:
import os
import openpyxl
import re
import pandas as pd

# Define a function to remove all leading and trailing whitespace
def remove_whitespace(text):
    return re.sub(r'^\s+|\s+$', '', text)

# Function to find the header row and extract information from a single Excel file
# Function to extract information from an Excel file
def extract_info_from_excel(file_path):
    try:
        if file_path.endswith('.xlsx'):
            workbook = openpyxl.load_workbook(file_path)
            sheet = workbook.active
            df = pd.read_excel(file_path, header=None)
            #print(df)

            data = []

            header_row_index = None
            code_col_index = None
            description_col_index = None
            unit_col_index = None
            
            # Find the header row using the same keywords as before
            header_row_index = None
            for i, row in enumerate(df.values):
                for j, cell_value in enumerate(row):
                    if cell_value is not None and any(keyword in str(cell_value).lower() for keyword in
                                                      ["artigo", "cap", "capitulo", "codigo", "descrição",
                                                       "designação"]):
                        header_row_index = i
                        break
                if header_row_index is not None:
                    break

            if header_row_index is None:
                print(f"No header row found in {file_path}")
                return []

            header_row = df.iloc[header_row_index]
            print(header_row)

            # Find the column indices for code, description, and unit
            code_col = None
            description_col = None
            unit_col = None
            unit_pattern = r'\bun\b'

            for i, header in enumerate(header_row):
                if header and any(keyword in header.lower() for keyword in ["art", "cap", "capitulo", "codigo", "artigo","código"]):
                    code_col = i
                    print("code col")
                    print(code_col)
                elif header and any(keyword in header.lower() for keyword in ["descrição", "designação"]):
                    description_col = i
                    print("descrip index col")
                    print(description_col)
                elif header and any(keyword in header.lower() for keyword in ["un ", "unidade", "un.", "un()"]):
                    unit_col = i
                    print("unit index col")
                    print(unit_col)

            # Initialize variables to store previous values
            prev_code = None
            prev_description = None
            prev_unit = None

            # Extract data based on column indices
            # Check if the header row index is found
            if header_row_index is not None:
                # Initialize variables to store previous values
                prev_code = None
                prev_description = None
                prev_unit = None

                # Extract data based on column indices
                for row in sheet.iter_rows(min_row=header_row_index + 1, values_only=True):
                    print(code_col)
                    code = row[code_col] if code_col else ""
                    print(code)
                    description = row[description_col] if description_col else ""
                    print(description)
                    unit = row[unit_col] if unit_col else ""

                    # Check if the row contains meaningful data (not None and not "")
                    if any(item is not None and item != "" for item in [code, description, unit]):
                        # Check if the values have changed from the previous row
                        if code != prev_code or description != prev_description or unit != prev_unit:
                            # Count the number of None values
                            none_count = sum(1 for item in [code, description, unit] if item is None)
                            # Check if at least two values are not None; if not, skip the row
                            if none_count >= 2:
                                continue
                            data.append((code, description, unit))
                        # Append the data to the DataFrame, filtering out rows with two None values

                        # Update the previous values for the next iteration
                        prev_code = code
                        prev_description = description
                        prev_unit = unit
            else:
                print(f"No header row found in {file_path}")

            return data

        elif file_path.endswith('.xls'):
            # Use pandas to read XLS file
            df = pd.read_excel(file_path, header=None)
            
            # Find the header row using the same keywords as before
            header_row_index = None
            for i, row in enumerate(df.values):
                for j, cell_value in enumerate(row):
                    if cell_value is not None and any(keyword in str(cell_value).lower() for keyword in
                                                      ["artigo", "cap", "capitulo", "codigo", "descrição",
                                                       "designação"]):
                        header_row_index = i
                        break
                if header_row_index is not None:
                    break

            if header_row_index is None:
                print(f"No header row found in {file_path}")
                return []

            header_row = df.iloc[header_row_index]
            #print("-------")
            #print(header_row)
            data = []

            code_col_index = None
            description_col_index = None
            unit_col_index = None
            pattern_a = r'\b(?!\s)(?:art|cap|codigo|artigo)\b(?!\w)'
            pattern_b= r'\b(?:un|unidade|un.)\b'
            for i, header in enumerate(header_row):
                if isinstance(header, str):  # Check if header is a string
                    header = remove_whitespace(header)
                    header = re.sub(r'[\t\x0B\x0C\x00-\x08\x0E-\x1F\x7F-\x9F]+', ' ', header)
                    #print(header)
                    
                    if code_col_index is None:  # Check if code_col_index is not assigned yet
                        match = re.search(pattern_a, header.lower(), flags=re.IGNORECASE)
                        if match:
                            #print("asa")
                            code_col_index = i
                            #print(code_col_index)
                    
                    if header and any(keyword in str(header).lower() for keyword in ["descrição", "designação"]):
                        #print("asdsa")
                        description_col_index = i
                        #print(description_col_index)
                    
                    if re.search(pattern_b, header.lower(), flags=re.IGNORECASE):
                        #print("wqewqea")
                        unit_col_index = i
                        #print(unit_col_index)
            
            # Check the values of code_col_index, description_col_index, and unit_col_index here
            print("Code Column Index:", code_col_index)
            print("Description Column Index:", description_col_index)
            print("Unit Column Index:", unit_col_index)

            prev_code = None
            prev_description = None
            prev_unit = None

            for index, row in df.iloc[header_row_index+1:].iterrows():
                code = row[code_col_index] if code_col_index is not None else ""
                #print("code")
                #print(code)
                description = row[description_col_index] if description_col_index is not None else ""
                #print("descript")
                #print(description)
                unit = row[unit_col_index] if unit_col_index is not None else ""
                #print("unidade")
                #print(unit)

                # Check if at least two of the variables are not None, not empty strings, and not NaN
                if sum(1 for item in [code, description, unit] if pd.notna(item) and item != "") >= 2:
                    data.append((code, description, unit))
                    

                    # Update the previous values for the next iteration
                    prev_code = code
                    prev_description = description
                    prev_unit = unit

            return data

        else:
            print(f"Unsupported file format: {file_path}")
            return []

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return []

# Folder containing Excel files
folder_path = '/path/to/excel/files'

# List to store extracted data from all files
#all_data = []
# Create an empty DataFrame to store the data
#df = pd.DataFrame(columns=['Code', 'Description', 'Unit'])

# Iterate through Excel files in the folder
#for filename in os.listdir(folder_path):
#    if filename.endswith('.xlsx'):
#        file_path = os.path.join(folder_path, filename)
#        data = extract_info_from_excel(file_path)
#        all_data.extend(data)

# Print the extracted data
#for code, description, unit in all_data:
    #print(f"Code: {code}, Description: {description}, Unit: {unit}")

file_path = r"C:\Users\Luís\OneDrive - Universidade do Porto\Scrape de MQT data treatment experiments\All MQT Data\00.1 MQT.xlsx"
extract_info_from_excel(file_path)

0            Código
1        Designação
2               Un.
3           Quanto.
4    Preço unitário
5             Total
Name: 2, dtype: object
code col
0
descrip index col
1
unit index col
2
0

Designação
0

Notas:
I) Todas as marcas referidas deverão ser entendidas apenas como indicadoras das caraterísticas e propriedades dos produtos descritos, devendo ser sempre interpretadas como as marcas descritas ou equivalentes.
II) Nos casos em que materiais ou elementos de materiais são indicados pela sua marca, sem prejuízo de ser referido a sua equivalência, justifica-se pela impossibilidade da descrição  suficientemente esclarecedora, exaustiva, precisa e inteligível das características técnicas do material, razão pela qual é de todo vantajoso a referência à marca ou a produto equivalente.
III) Assim, no cumprimento do disposto no n.º 9 do art.º 49 do CCP, as referências mencionadas no número anterior só são autorizadas, a título excecional, devendo sempre ser acompanhada da menção "ou equ

[('', 'Designação', 'Un.'),
 ('',
  'Notas:\nI) Todas as marcas referidas deverão ser entendidas apenas como indicadoras das caraterísticas e propriedades dos produtos descritos, devendo ser sempre interpretadas como as marcas descritas ou equivalentes.\nII) Nos casos em que materiais ou elementos de materiais são indicados pela sua marca, sem prejuízo de ser referido a sua equivalência, justifica-se pela impossibilidade da descrição  suficientemente esclarecedora, exaustiva, precisa e inteligível das características técnicas do material, razão pela qual é de todo vantajoso a referência à marca ou a produto equivalente.\nIII) Assim, no cumprimento do disposto no n.º 9 do art.º 49 do CCP, as referências mencionadas no número anterior só são autorizadas, a título excecional, devendo sempre ser acompanhada da menção "ou equivalente".',
  None),
 ('', 'Trabalhos preparatórios', None),
 ('',
  'Montagem e desmontagem e manutenção de estaleiro de apoio à obra em local a definir pela fiscaliz